# 33. notMNIST CNN

<p style="text-align: right;">
blackdew7@gmail.com<br>
Your name :
</p>

### 텐서플로 실습시간 그 일곱번째!<br>

#### 선행지식
1. TensorFlow 다루기 기초
2. 모델링을 한다는 것에 대한 이해.
3. Supervised Learning 중 Classification에 대한 기본 지식.
4. CNN에 대한 기본 구조와 개념

#### 실습목표
1. CNN의 구조를 Graph로 그려낼 수 있다.
2. 그려낸 Graph를 텐서플로우를 이용해 코딩할 수 있다.

#### 사용데이터.

01. Multinomial Classification : http://yann.lecun.com/exdb/mnist/

## 00. 라이브러리 불러오기

In [ ]:
# 메모리 초기화
%reset

In [1]:
import sys
import tarfile

tar = tarfile.open('./data/notMNIST_small.tar.gz')
sys.stdout.flush()
tar.extractall('./data/')
tar.close()

In [ ]:
# MNIST and Convolutional Neural Network
import tensorflow as tf
import random
import numpy as np
import matplotlib.pyplot as plt

## 00. 이미지 로더

In [ ]:
!pip install imageio

In [ ]:
from image_loader import ImageLoader
mnist = ImageLoader('./data/notMNIST_small/', ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'])

### 이미지 로딩 테스트

In [ ]:
print(len(mnist.train.images))   # Images in the training set.
print(len(mnist.train.labels))   # Labels in the training set.
print(len(mnist.test.images))    # Images in the test set.
print(len(mnist.test.labels))    # Labels in the test set.

train, label = mnist.train.next_batch(100) # To retrieve specified number of data in a batch

print(np.array(train).shape)
print(np.array(label).shape)

## 01. Import Library & Prepare notMNIST Data

In [ ]:
import tensorflow as tf
from tbij import show_graph # Tensor Board In Jupyter notebook. You can see this later.

import numpy as np
import matplotlib.pyplot as plt

from image_loader import ImageLoader
mnist = ImageLoader('./data/notMNIST_small/', ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'])

## 02. Prepare Variables for a CNN Graph with TF

In [ ]:
# Graph Clear
tf.reset_default_graph()
tf.set_random_seed(2017) # random seeding - reproduct

############################
# Place Holders

X = tf.placeholder(tf.float32, [None, 28, 28], name="X")
X_img = tf.reshape(X, [-1, 28, 28, 1], name="X_img")
Y = tf.placeholder(tf.float32, [None, 10], name="Y")


## 03. Make a CNN Graph with TF

In [ ]:
#########################
# ConvLayer 01 

with tf.name_scope("layer1_conv"):

    # for Conv Layer 01 filter - shape=(3, 3, 1)
    W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01), name="W1")

    # Convolution Layer 01 -> (?, 28, 28, 32)
    L1_conv = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME', name="L1_conv")
    L1_conv = tf.nn.relu(L1_conv, name="L1_relu")

    # Pooling Layer 01 -> (?, 14, 14, 32)
    L1_pool = tf.nn.max_pool(L1_conv, ksize=[1, 2, 2, 1], 
                             strides=[1, 2, 2, 1], padding='SAME', name="L1_max_pool")

    
#########################
# ConvLayer 02

with tf.name_scope("layer2_conv"):

    # for Conv Layer 02 filter - shape=(3, 3, 32)
    W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01), name="W2")

    # Convolution Layer 02 -> (?, 14, 14, 64)
    L2_conv = tf.nn.conv2d(L1_pool, W2, strides=[1, 1, 1, 1], padding='SAME', name="L2_conv")
    L2_conv = tf.nn.relu(L2_conv, name="L2_relu")

    # Pooling Layer 02 -> (?, 7, 7, 64)
    L2_pool = tf.nn.max_pool(L2_conv, ksize=[1, 2, 2, 1], 
                             strides=[1, 2, 2, 1], padding='SAME', name="L2_max_pool")

    
#########################
# Fully Connected Layer 01

with tf.name_scope("layer1_fc"):

    ###############################
    # Flatten layer
    L_flat = tf.reshape(L2_pool, [-1, 7 * 7 * 64], name="L_flat")

    # for Final FC 7x7x64 inputs -> 1000
    W1_fc = tf.Variable(tf.random_normal([7 * 7 * 64, 1000], stddev=0.01), name="W1_fc")

    # for Final FC Layer 1: 7x7x64 inputs -> 1000
    L1_fc = tf.matmul(L_flat, W1_fc, name="L1_fc")
    L1_fc = tf.nn.relu(L1_fc, name="L1_fc_relu")

    #########################
    # Dropout
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    L1_fc_dropout = tf.nn.dropout(L1_fc, keep_prob=keep_prob, name="dropout")
    #########################
    
    
#########################
# Fully Connected Layer 02

with tf.name_scope("layer2_fc"):

    # for Final FC 1000 inputs -> 10
    W2_fc = tf.Variable(tf.random_normal([1000, 10], stddev=0.01), name="W2_fc")

    # bias
    b = tf.Variable(tf.random_normal([10]), name="bias")

    # for Final FC Layer 2: 1000 inputs -> 10
    logits = tf.add(tf.matmul(L1_fc_dropout, W2_fc), b, name="logits")

    
#########################
# Cost & Optimizer

learning_rate = 0.001

with tf.name_scope("Optimizer"):
    # Cost(loss) function & Optimizer
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y, name="cross_entropy")
    cost = tf.reduce_mean(cross_entropy, name="cost")
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

###########################
# Evaluation
with tf.name_scope("Prediction"):
    # Test model and check accuracy
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1), name="compare")
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")

## 04. Train & Evaluation

In [ ]:
#########################
# Session initialize

sess = tf.Session()
sess.run(tf.global_variables_initializer())

###########################
# Traning

print('Learning started. It takes sometime.')

epochs = 1
batch_size = 500

for epoch in range(epochs):
    print("%dth epoch" % (epoch + 1))

    # 총 갯수: 15000여개를 500개씩 (batch_size) 나누어 훈련 
    #        => 30번 (n_of_batches) 훈련하게 된다. 
    n_of_batches = int(len(mnist.train.images) / batch_size)
    
    for i in range(n_of_batches):
        X_batch, Y_batch = mnist.train.next_batch(batch_size)
        # 학습 진행
        sess.run(optimizer, feed_dict={X: X_batch, Y: Y_batch, keep_prob: 0.7})
        
        if ((i + 1) % 2 == 0):
            # 학습 상황 디스플레이
            loss = sess.run(cost, feed_dict={X: X_batch, Y: Y_batch, keep_prob: 0.7})
            print("%dth records, training cost: %.3f" % (((i + 1) * batch_size), loss))
            
print("Training Complete")

print('Accuracy: %.2f' % (sess.run(accuracy, feed_dict={X: mnist.test.images, 
                                                        Y: mnist.test.labels, 
                                                        keep_prob: 1.0}) * 100))

#### 그래프를 그려보자

In [ ]:
from tbij import show_graph # Tensor Board In Jupyter notebook. You can see this later.
show_graph(tf.get_default_graph().as_graph_def())

### 성능을 한 번 확인해 보자. 제대로 되고 있는걸까?

In [ ]:
# Get one and predict
labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
r = random.randint(0, len(mnist.test.images) - 1)

y = sess.run(tf.argmax(np.reshape(mnist.test.labels[r:r + 1], (1, 10)), 1))
y_pred = sess.run(tf.argmax(logits, 1), 
                  feed_dict={X: np.reshape(mnist.test.images[r:r + 1], (-1, 28, 28)), 
                             keep_prob: 1.0})

print("Label: ", labels[int(y)])
print("Prediction: ", labels[int(y_pred)])

import matplotlib.pyplot as plt
plt.imshow(np.reshape(mnist.test.images[r:r + 1], (28, 28)), cmap='Greys', interpolation='nearest')
plt.show()

## 05. More Deep

In [ ]:
import tensorflow as tf
from tbij import show_graph # Tensor Board In Jupyter notebook. You can see this later.

import numpy as np
import matplotlib.pyplot as plt

from image_loader import ImageLoader
mnist = ImageLoader('./data/notMNIST_small/', ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'])

In [ ]:
# Graph Clear
tf.reset_default_graph()
tf.set_random_seed(2017) # random seeding - reproduct

############################
# Place Holders

X = tf.placeholder(tf.float32, [None, 28, 28], name="X")
X_img = tf.reshape(X, [-1, 28, 28, 1], name="X_img")
Y = tf.placeholder(tf.float32, [None, 10], name="Y")


In [ ]:
#########################
# ConvLayer 01, 02

with tf.name_scope("layer1_conv"):

    # for Conv Layer 01 filter - shape=(3, 3, 1)
    W1 = tf.Variable(tf.random_normal([3, 3, 1, 16], stddev=0.01),  name="W1")

    # Convolution Layer 01 -> (?, 28, 28, 16)
    L1_conv = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME', name="L1_conv")
    L1_conv = tf.nn.relu(L1_conv, name="L1_relu")

    # for Conv Layer 01 filter - shape=(3, 3, 16)
    W2 = tf.Variable(tf.random_normal([3, 3, 16, 32], stddev=0.01), name="W2")

    # Convolution Layer 01 -> (?, 28, 28, 32)
    L2_conv = tf.nn.conv2d(L1_conv, W2, strides=[1, 1, 1, 1], padding='SAME', name="L2_conv")
    L2_conv = tf.nn.relu(L2_conv, name="L2_relu")
    
    # Pooling Layer 01 -> (?, 14, 14, 32)
    L12_pool = tf.nn.max_pool(L2_conv, ksize=[1, 2, 2, 1], 
                              strides=[1, 2, 2, 1], padding='SAME', name="L12_max_pool")

    
#########################
# ConvLayer 03, 04

with tf.name_scope("layer2_conv"):

    # for Conv Layer 02 filter - shape=(3, 3, 32)
    W3 = tf.Variable(tf.random_normal([3, 3, 32, 48], stddev=0.01), name="W3")

    # Convolution Layer 02 -> (?, 14, 14, 32)
    L3_conv = tf.nn.conv2d(L12_pool, W3, strides=[1, 1, 1, 1], padding='SAME', name="L3_conv")
    L3_conv = tf.nn.relu(L3_conv, name="L3_relu")

    # for Conv Layer 02 filter - shape=(3, 3, 48)
    W4 = tf.Variable(tf.random_normal([3, 3, 48, 64], stddev=0.01), name="W4")

    # Convolution Layer 02 -> (?, 14, 14, 64)
    L4_conv = tf.nn.conv2d(L3_conv, W4, strides=[1, 1, 1, 1], padding='SAME', name="L4_conv")
    L4_conv = tf.nn.relu(L4_conv, name="L4_relu")

    # Pooling Layer 02 -> (?, 7, 7, 64)
    L34_pool = tf.nn.max_pool(L4_conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name="L34_max_pool")

    
#########################
# Fully Connected Layer 01

with tf.name_scope("layer1_fc"):

    ###############################
    # Flatten layer
    L_flat = tf.reshape(L34_pool, [-1, 7 * 7 * 64], name="L_flat")

    # for Final FC 7x7x64 inputs -> 1000
    W1_fc = tf.Variable(tf.random_normal([7 * 7 * 64, 1000], stddev=0.01), name="W1_fc")

    # for Final FC Layer 1: 7x7x64 inputs -> 1000
    L1_fc = tf.matmul(L_flat, W1_fc, name="L1_fc")
    L1_fc = tf.nn.relu(L1_fc, name="L1_fc_relu")

    #########################
    # Dropout
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    L1_fc_dropout = tf.nn.dropout(L1_fc, keep_prob=keep_prob, name="dropout")
    #########################
    
    
#########################
# Fully Connected Layer 02

with tf.name_scope("layer2_fc"):

    # for Final FC 1000 inputs -> 10
    W2_fc = tf.Variable(tf.random_normal([1000, 10], stddev=0.01), name="W2_fc")

    # bias
    b = tf.Variable(tf.random_normal([10]), name="bias")

    # for Final FC Layer 2: 1000 inputs -> 10
    logits = tf.add(tf.matmul(L1_fc_dropout, W2_fc), b, name="logits")

    
#########################
# Cost & Optimizer

learning_rate = 0.001

with tf.name_scope("Optimizer"):
    # Cost(loss) function & Optimizer
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y, name="cross_entropy")
    cost = tf.reduce_mean(cross_entropy, name="cost")
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


###########################
# Evaluation
with tf.name_scope("Prediction"):
    # Test model and check accuracy
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1), name="compare")
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")

In [ ]:
#########################
# Session initialize

sess = tf.Session()
sess.run(tf.global_variables_initializer())

###########################
# Traning

print('Learning started. It takes sometime.')

epochs = 10
batch_size = 500

for epoch in range(epochs):
    print("%dth epoch" % (epoch + 1))

    # 총 갯수: 55000개를 500개씩 (batch_size) 나누어 훈련 
    #        => 110번 (n_of_batches) 훈련하게 된다. 
    n_of_batches = int(len(mnist.train.images) / batch_size)
    
    for i in range(n_of_batches):
        X_batch, Y_batch = mnist.train.next_batch(batch_size)
        # 학습 진행
        sess.run(optimizer, feed_dict={X: X_batch, Y: Y_batch, keep_prob: 0.7})
        
        if ((i + 1) % 2 == 0):
            # 학습 상황 디스플레이
            loss = sess.run(cost, feed_dict={X: X_batch, Y: Y_batch, keep_prob: 0.7})
            print("%dth records, training cost: %.3f" % (((i + 1) * batch_size), loss))
        
print("Training Complete")
    
print('Accuracy: %.2f' % (sess.run(accuracy, feed_dict={X: mnist.test.images, 
                                                        Y: mnist.test.labels, 
                                                        keep_prob: 1.0}) * 100))

#### 그래프를 그려보자

In [ ]:
from tbij import show_graph # Tensor Board In Jupyter notebook. You can see this later.
show_graph(tf.get_default_graph().as_graph_def())

## 06. 대체 뭘 학습한 것인가?

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import math
import numpy as np

def plot_conv_weights(weights, input_channel=0):
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        w = sess.run(weights)

        w_min = np.min(w)
        w_max = np.max(w)

        num_filters = w.shape[3]
        num_grids = math.ceil(math.sqrt(num_filters))
        fig, axes = plt.subplots(num_grids, num_grids)

        for i, ax in enumerate(axes.flat):
            if i < num_filters:
                img = w[:, :, input_channel, i]
                ax.imshow(img, vmin=w_min, vmax=w_max, interpolation='nearest', cmap='seismic')

            ax.set_xticks([])
            ax.set_yticks([])

        plt.show()

In [ ]:
plot_conv_weights(weights=W1)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import math
import numpy as np

def plot_conv_layer(layer, image):

    with tf.Session() as sess:

        sess.run(tf.global_variables_initializer())

        feed_dict = {X: [image]}
        values = sess.run(layer, feed_dict=feed_dict)

        num_filters = values.shape[3]
        num_grids = math.ceil(math.sqrt(num_filters))
        fig, axes = plt.subplots(num_grids, num_grids)

        for i, ax in enumerate(axes.flat):
            if i < num_filters:
                img = values[0, :, :, i]
                ax.imshow(img, interpolation='nearest', cmap='binary')
                
            ax.set_xticks([])
            ax.set_yticks([])

        plt.show()

In [ ]:
img_shape = (28, 28)

def plot_image(image):
    plt.imshow(image.reshape(img_shape),
               interpolation='nearest',
               cmap='binary')

    plt.show()
    
image1 = mnist.test.images[0]
plot_image(image1)

image2 = mnist.test.images[1]
plot_image(image2)

In [ ]:
plot_conv_layer(layer=L2_pool, image=image1)

In [ ]:
# 세션을 종료하자
sess.close()